In [1]:
#导入
import pandas as pd
pd.reset_option('display.float_format')
pd.options.display.float_format = '{:.2f}'.format
from openpyxl import load_workbook

df_ffca= pd.read_excel(r"C:\Users\Yuyan.Lu\Desktop\BB REPORT TEST FILE\0118\bb_doc\FFCA.xlsx")
df_qv = pd.read_excel(r"C:\Users\Yuyan.Lu\Desktop\BB REPORT TEST FILE\0118\bb_doc\QV.xlsx")
df_stock = pd.read_excel(r"C:\Users\Yuyan.Lu\Desktop\BB REPORT TEST FILE\0118\bb_doc\stock.xlsx") 
df_stock.drop([0])
df_inventory = pd.read_excel(r"C:\Users\Yuyan.Lu\Desktop\BB REPORT TEST FILE\0118\bb_doc\inventory.xlsx")
df_CS = pd.read_excel(r"C:\Users\Yuyan.Lu\Desktop\BB REPORT TEST FILE\0118\bb_doc\CS.xlsx")
df_CP = pd.read_excel(r"C:\Users\Yuyan.Lu\Desktop\BB REPORT TEST FILE\0118\bb_doc\fixed_doc\CP_approved.xlsx")
df_movement = pd.read_excel(r"C:\Users\Yuyan.Lu\Desktop\BB REPORT TEST FILE\0118\bb_doc\Movement.xlsx")
# df_WC = pd.read_excel(r"C:\Users\Yuyan.Lu\Desktop\BB REPORT TEST FILE\0118\bb_doc\Warehouse_Risk_Category.xlsx")
df_vessel = pd.read_excel(r"C:\Users\Yuyan.Lu\Desktop\BB REPORT TEST FILE\0118\bb_doc\inventory2.xlsx")
df_invoice = pd.read_excel(r"C:\Users\Yuyan.Lu\Desktop\BB REPORT TEST FILE\0118\bb_doc\invoice.xlsx")
input= pd.read_excel(r"C:\Users\Yuyan.Lu\Desktop\BB REPORT TEST FILE\0118\bb_doc\input.xlsx")

In [60]:
#filter

cols_ffca_1 = ['Purchase Assignment ID', 'Inventory Status']
cols_input = ['assignment']

df_ffca_filter = pd.DataFrame(df_ffca, columns=cols_ffca_1)
df_input = pd.DataFrame(input, columns=cols_input)

assignment_filter = pd.merge(df_ffca_filter, df_input, left_on=["Purchase Assignment ID"], right_on=["assignment"], how= 'inner')
assignment_filter.columns = ['Purchase Assignment Reference', 'Inventory Status', 'Assignment']

def expected(a):
    return a == "Expected"
def instorage(a):
    return a == "In-Storage"
def intransit(a):
    return a == "In-Transit"
def delivered(a):
    return a == "Delivered"

df1 = assignment_filter.loc[assignment_filter['Inventory Status'].apply(expected)]
df2 = assignment_filter.loc[assignment_filter['Inventory Status'].apply(instorage)]
df3 = assignment_filter.loc[assignment_filter['Inventory Status'].apply(intransit)]
df4 = assignment_filter.loc[assignment_filter['Inventory Status'].apply(delivered)]

In [61]:
#sales

#column filter
cols_ffca = ['Purchase Assignment ID', 'Sales Assignment ID ', 'Commodity', 'Inventory ID','Counterparty of Purchase', 'Counterparty of Sales', 'Sales Invoice ID', 'Estimated Sales Valuation', \
    'Purchase Settled Amount', 'Authorised Paid Quantity', 'Financed Quantity', 'Current Value', 'Treasury Payment Date', 'Purchase Invoice', 'Purchase Invoiced Amount', \
        'Sales Invoiced Amount', 'Sales Settled Amount']
cols_qv = ['Assignment ID', 'Group Company', 'Trade ID', 'Quota ID', 'Traffic Operator']
cols_inventory = ['INVENTORY ID', 'PURCHASE ASSIGNMENT', 'DRY WEIGHT', 'WET WEIGHT', 'EXPECTED RECEIPT DATE']
cols_stock = ['Inventory Id','Origin Country']
cols_cs = ['ID', 'Assignments', 'Term', 'Issuing/Collecting Bank', '1st Confirming Bank', 'Opening Date', 'Expiry Date', 'Balance Amount', 'Maximum Amount']
cols_ffca_blend = ['Purchase Assignment ID', 'Sales Assignment ID ', 'Commodity', 'Inventory ID','Counterparty of Purchase', 'Counterparty of Sales', 'Sales Invoice ID', 'Estimated Sales Valuation', \
    'Purchase Settled Amount', 'Authorised Paid Quantity', 'Financed Quantity', 'Current Value', 'Treasury Payment Date', 'Purchase Invoice', 'Purchase Invoiced Amount', \
        'Sales Invoiced Amount', 'Sales Settled Amount', 'QTY']
cols_invoice = ['Assignment Ref', 'Invoice Type', 'Invoice Date', 'Payment Expected Due Date', 'Days Overdue', 'Invoice No', 'Unpaid Amount USD', 'Invoice Amount']

df_ffca_f1 = pd.DataFrame(df_ffca, columns=cols_ffca)
df_qv_f1 = pd.DataFrame(df_qv, columns=cols_qv)
df_inventory_f1 = pd.DataFrame(df_inventory, columns=cols_inventory)
df_stock_f1 = pd.DataFrame(df_stock, columns=cols_stock)
df_CS_f1 = pd.DataFrame(df_CS, columns=cols_cs)
df_invoice_f1 = pd.DataFrame(df_invoice, columns=cols_invoice)
df_ffca_f1['Current Value'] = pd.to_numeric(df_ffca_f1['Current Value'], errors='coerce')
df_ffca_f1['Authorised Paid Quantity'] = pd.to_numeric(df_ffca_f1['Authorised Paid Quantity'], errors='coerce')
df_ffca_f1['Financed Quantity'] = pd.to_numeric(df_ffca_f1['Financed Quantity'], errors='coerce')


#sales
#delete nan sales
df_ffca_f1.dropna(subset=['Sales Assignment ID '], inplace=True)
# 一、先将字段拆分
df_CS_f1['Assignments']=df_CS['Assignments'].map(lambda x:x.split(','))
#二、然后直接调用explode()方法
df_CS_f1_explode = df_CS_f1.explode('Assignments')

def blend(a):
    return a == "Blend"

#blend
bdf_ffca_f1 = pd.DataFrame(df_ffca, columns=cols_ffca_blend)
df_blend_inventory = df_inventory_f1.loc[df_inventory_f1['PURCHASE ASSIGNMENT'].apply(blend)]

row_list = df_inventory_f1[df_inventory_f1['PURCHASE ASSIGNMENT'] == 'Blend'].index.tolist() # 获得含有该值的行的行号
df_inventory_f1 = df_inventory_f1.drop(row_list)

bffca_qv = pd.merge(bdf_ffca_f1, df_qv_f1, left_on=["Purchase Assignment ID"], right_on=["Assignment ID"], how= 'inner')
bffca_qv_cp = pd.merge(bffca_qv, df_CP, left_on=['Counterparty of Sales'], right_on=['Sales Assignment Counterparty'], how='outer')
binventory_stock = pd.merge(df_blend_inventory, df_stock_f1, left_on=['INVENTORY ID'], right_on=['Inventory Id'])
FQC = pd.merge(bffca_qv_cp, df_CS_f1_explode, left_on=["Sales Assignment ID "], right_on=["Assignments"], how = 'outer')
bfinal_sales = pd.merge(FQC, binventory_stock, left_on=['Inventory ID'], right_on=['INVENTORY ID'])

def function(a, b, c):
    return c * (1 + (a-b)/a)

bfinal_sales['Purchased Wet Weight']  = bfinal_sales.apply(lambda x: function(x['WET WEIGHT'], x['DRY WEIGHT'], x['QTY']), axis = 1)

bfinal_sales = bfinal_sales.drop(columns = ['WET WEIGHT', 'DRY WEIGHT'])


ffca_qv = pd.merge(df_ffca_f1, df_qv_f1, left_on=["Purchase Assignment ID"], right_on=["Assignment ID"], how= 'inner')
ffca_qv_cp = pd.merge(ffca_qv, df_CP, left_on=['Counterparty of Sales'], right_on=['Sales Assignment Counterparty'], how='outer')
inventory_stock = pd.merge(df_inventory_f1, df_stock_f1, left_on=['INVENTORY ID'], right_on=['Inventory Id'])
nfinal_sales = pd.merge(FQC, inventory_stock, left_on=['Inventory ID'], right_on=['INVENTORY ID'])
nfinal_sales.columns = ['Purchase Assignment ID', 'Sales Assignment ID ', 'Commodity',
       'Inventory ID', 'Counterparty of Purchase', 'Counterparty of Sales',
       'Sales Invoice ID', 'Estimated Sales Valuation',
       'Purchase Settled Amount', 'Authorised Paid Quantity',
       'Financed Quantity', 'Current Value', 'Treasury Payment Date',
       'Purchase Invoice', 'Purchase Invoiced Amount', 'Sales Invoiced Amount',
       'Sales Settled Amount', 'QTY', 'Assignment ID', 'Group Company',
       'Trade ID', 'Quota ID', 'Traffic Operator',
       'Sales Assignment Counterparty',
       'Sales Contractual Counterparty Open Account Approved ', 'ID',
       'Assignments', 'Term', 'Issuing/Collecting Bank', '1st Confirming Bank',
       'Opening Date', 'Expiry Date', 'Balance Amount', 'Maximum Amount',
       'INVENTORY ID', 'PURCHASE ASSIGNMENT', 'Purchase Dry Weight', 'Purchase Wet Weight',
       'EXPECTED RECEIPT DATE', 'Inventory Id', 'Origin Country']

order = ['Purchase Assignment ID', 'Sales Assignment ID ', 'Commodity','Inventory ID', 'Counterparty of Purchase', 'Counterparty of Sales', 'Sales Invoice ID', 'Estimated Sales Valuation', \
       'Purchase Settled Amount', 'Authorised Paid Quantity','Financed Quantity', 'Current Value', 'Treasury Payment Date','Purchase Invoice', 'Purchase Invoiced Amount', 'Sales Invoiced Amount','Sales Settled Amount',  \
        'Assignment ID', 'Group Company','Trade ID', 'Quota ID', 'Traffic Operator','Sales Assignment Counterparty','Sales Contractual Counterparty Open Account Approved ', 'ID', \
       'Assignments', 'Term', 'Issuing/Collecting Bank', '1st Confirming Bank','Opening Date', 'Expiry Date', 'Balance Amount', 'Maximum Amount', 'INVENTORY ID', 'PURCHASE ASSIGNMENT', 'QTY', 'Purchased Wet Weight',\
       'EXPECTED RECEIPT DATE', 'Inventory Id', 'Origin Country']

bfinal_sales = bfinal_sales[order]

bfinal_sales.columns = ['Purchase Assignment ID', 'Sales Assignment ID ', 'Commodity',
       'Inventory ID', 'Counterparty of Purchase', 'Counterparty of Sales',
       'Sales Invoice ID', 'Estimated Sales Valuation',
       'Purchase Settled Amount', 'Authorised Paid Quantity',
       'Financed Quantity', 'Current Value', 'Treasury Payment Date',
       'Purchase Invoice', 'Purchase Invoiced Amount', 'Sales Invoiced Amount',
       'Sales Settled Amount', 'Assignment ID', 'Group Company', 'Trade ID',
       'Quota ID', 'Traffic Operator', 'Sales Assignment Counterparty',
       'Sales Contractual Counterparty Open Account Approved ', 'ID',
       'Assignments', 'Term', 'Issuing/Collecting Bank', '1st Confirming Bank',
       'Opening Date', 'Expiry Date', 'Balance Amount', 'Maximum Amount',
       'INVENTORY ID', 'PURCHASE ASSIGNMENT', 'Purchase Dry Weight', 'Purchase Wet Weight',
       'EXPECTED RECEIPT DATE', 'Inventory Id', 'Origin Country']

final_sales = pd.concat([nfinal_sales,bfinal_sales])


In [62]:
final_sales = pd.merge(final_sales, df_invoice_f1, left_on=["Sales Assignment ID "], right_on=["Assignment Ref"])
# final_sales['Unpaid Amount USD'] = pd.to_numeric(final_sales['Unpaid Amount USD'], errors='coerce')
# final_sales['Invoice Amount'] = pd.to_numeric(final_sales['Invoice Amount'], errors='coerce')

df1 = final_sales[(final_sales['Sales Assignment ID '] == '670639.12.1')]


In [63]:
final_sales['Current Value'] = pd.to_numeric(final_sales['Current Value'], errors='coerce')

#Comparison between invoice amount and current value


# def function1(a,b,c):
#     if 3 * (-b) >= a:
#         return (-b) - c
#     else:
#         return 0

def function1(a,b):
    if a == 'PROVISIONAL':
        return b
    else:
        return 0

# def function2(a,b,c):
#     if 3 * (-b) < a:
#         return (-b) - c
#     else:
#         return 0

def function2(a,b):
    if a == 'FINAL':
        return b
    else:
        return 0

final_sales['Unpaid Provisional Receivables(USD)'] = final_sales.apply(lambda x: function1(x['Invoice Type'], x['Unpaid Amount USD']), axis = 1)
final_sales['Unpaid Final Receivables(USD)'] = final_sales.apply(lambda x: function2(x['Invoice Type'], x['Unpaid Amount USD']), axis = 1) #wrong
final_sales.loc[:,"Uninvoiced Final Receivables(USD)"] = final_sales["Estimated Sales Valuation"] - final_sales["Invoice Amount"]


In [64]:
final_sales['Current Value'] = pd.to_numeric(final_sales['Current Value'], errors='coerce')

#Comparison between invoice amount and current value


# def function1(a,b,c):
#     if 3 * (-b) >= a:
#         return (-b) - c
#     else:
#         return 0

def function1(a,b):
    if a == 'PROVISIONAL':
        return b
    else:
        return 0

# def function2(a,b,c):
#     if 3 * (-b) < a:
#         return (-b) - c
#     else:
#         return 0

def function2(a,b):
    if a == 'FINAL':
        return b
    else:
        return 0

final_sales['Unpaid Provisional Receivables(USD)'] = final_sales.apply(lambda x: function1(x['Invoice Type'], x['Unpaid Amount USD']), axis = 1)
final_sales['Unpaid Final Receivables(USD)'] = final_sales.apply(lambda x: function2(x['Invoice Type'], x['Unpaid Amount USD']), axis = 1) #wrong
final_sales.loc[:,"Uninvoiced Final Receivables(USD)"] = final_sales["Estimated Sales Valuation"] - final_sales["Invoice Amount"]

# final_sales.loc[:,"SCR Covered By LC (SCR-Sales Contractual Rights)"] = final_sales["Current Value"] - final_sales["Sales Invoiced Amount"]

#Comparison between invoice amount and LC current balance
def function3(a,b):
    if -a >= b:
        return b
    if -a < b:
        return -a

def function4(a,b):
    if -a < b:
        return b + a
    if -a >= b:
        return 0

def function5(a,b):
    if a >= b:
        return b
    else:
        return a

def function6(a):
    if a == "FINAL":
        return "100%"
    else:
        return "90%"


final_sales['Total Financeable Sales Invoices Covered by LC']  = final_sales.apply(lambda x: function3(x['Invoice Amount'], x['Balance Amount']), axis = 1)
final_sales['LC Cover Available to SCR (SCR-Sales Contractual Rights)']  = final_sales.apply(lambda x: function4(x['Invoice Amount'], x['Balance Amount']), axis = 1)
final_sales['Sales Provisional Percentage']  = final_sales.apply(lambda x: function6(x['Invoice Type']), axis = 1)
final_sales['SCR Covered By LC (SCR-Sales Contractual Rights)']  = final_sales.apply(lambda x: function5(x['LC Cover Available to SCR (SCR-Sales Contractual Rights)'], x['Uninvoiced Final Receivables(USD)']), axis = 1)


import numpy as np

#adding 
final_sales['Sales Provisional Number'] = '1'
final_sales['Authorised Paid Quantity'] = pd.to_numeric(final_sales['Authorised Paid Quantity'], errors='coerce')
final_sales['Financed Quantity'] = pd.to_numeric(final_sales['Financed Quantity'], errors='coerce')
final_sales.loc[:,"Financing Ratio"] = final_sales["Authorised Paid Quantity"]/final_sales["Financed Quantity"]

final_sales['Cost Code ID'] = 'I01'
final_sales['Invoice Line Quantity'] = '0'
final_sales['Invoice Line Quantity UOM'] = 'MT'
final_sales['Purchase Dry Weight UOM'] = 'MT'
final_sales['Purchase Wet Weight UOM'] = 'MT'
final_sales['Receipt In Progress'] = 'N'
final_sales['Delivery In Progress'] = 'N'
final_sales['Summary Outstanding Receivables(USD)'] = final_sales['Unpaid Provisional Receivables(USD)'] + final_sales['Unpaid Final Receivables(USD)'] + final_sales['Uninvoiced Final Receivables(USD)']
final_sales['Sales Invoice Counterparty'] = final_sales['Counterparty of Sales']
final_sales['Eligible Dry Weight'] = final_sales['Purchase Dry Weight']
final_sales['Eligible Wet Weight'] = final_sales['Purchase Wet Weight']
final_sales['Paid Amount'] = final_sales['Invoice Amount'] - final_sales['Unpaid Amount USD']

def function7(a, b):
    if b == 0:
        return 0
    else:
        return -a/b
        
def function8(a):
    if a>1:
        return 1
    else:
        return a
        


final_sales['Sales Assignment Eligible Percentage']  = final_sales.apply(lambda x: function7(x['Estimated Sales Valuation'], x['Invoice Amount']), axis = 1)

final_sales['Sales Assignment Eligible Percentage']  = final_sales.apply(lambda x: function8(x['Sales Assignment Eligible Percentage']), axis = 1)




In [65]:
final_sales = final_sales.drop(columns=['Inventory ID', 'Current Value','Purchase Settled Amount', 'Purchase Invoice', 'Purchase Invoiced Amount', 'Current Value', 'Treasury Payment Date', \
    'Assignment ID', 'Sales Assignment Counterparty', 'Assignments', 'INVENTORY ID', 'PURCHASE ASSIGNMENT', 'Unpaid Amount USD', \
        'EXPECTED RECEIPT DATE', 'Inventory Id', 'Origin Country', 'QTY', 'Assignment Ref', 'Invoice No', 'Sales Invoiced Amount', 'Sales Settled Amount'], axis=1)

In [66]:


final_sales.columns = ['Purchase Assignment Reference', 'Sales Assignment Reference', 'Purchase Commodity',
       'Purchase Contractual Counterparty', 'Sales Assignment Counterparty', 'Sales Invoice ID',
       'Cost Amount(USD)', 'Authorised Paid Quantity','Financed Quantity', 
       'Purchase Group Company Code', 'Purchase Trade ID', 'Purchase Quota Titan ID', 'Purchase Traffic Operator',
       'Sales Contractual Counterparty Open Account Approved', 'LC Reference', 'Sales Credit Security Type',
       'LC Opening Bank', 'LC Confirming Bank', 'LC Issuance Date', 'LC Expiry Date', 'LC Current Balance', 'LC Original Amount',
       'Purchase Dry Weight', 'Purchase Wet Weight', 
       'Sales Provisional Final', 'Sales Invoice Date', 'Sales Invoice Due Date', 'Sales Invoice Days Overdue','Invoice Line Amount(USD)', 'Unpaid Provisional Receivables(USD)',
       'Unpaid Final Receivables(USD)', 'Uninvoiced Final Receivables(USD)',
       'Total Financeable Sales Invoices Covered by LC',
       'LC Cover Available to SCR (SCR-Sales Contractual Rights)',
       'Sales Provisional Percentage',
       'SCR Covered By LC (SCR-Sales Contractual Rights)',
       'Sales Provisional Number', 'Financing Ratio', 'Cost Code ID',
       'Invoice Line Quantity', 'Invoice Line Quantity UOM',
       'Purchase Dry Weight UOM', 'Purchase Wet Weight UOM',
       'Receipt In Progress', 'Delivery In Progress',
       'Summary Outstanding Receivables(USD)', 'Sales Invoice Counterparty',
       'Eligible Dry Weight', 'Eligible Wet Weight','Paid Amount',
       'Sales Assignment Eligible Percentage']


# final_sales.columns = ['Purchase Assignment Reference', 'Sales Assignment Reference', 'Purchase Commodity', 'Purchase Contractual Counterparty', \
#     'Sales Assignment Counterparty', 'Sales Invoice ID', 'Cost Amount(USD)', 'Authorised Paid Quantity', 'Financed Quantity', 'Invoice Line Amount(USD)', 'Paid Amount', 'Purchase Group Company Code', \
#         'Purchase Trade ID', 'Purchase Quota Titan ID', 'Purchase Traffic Operator', 'Sales Contractual Counterparty Open Account Approved', 'LC Reference', \
#             'Sales Credit Security Type','LC Opening Bank', 'LC Confirming Bank', 'LC Issuance Date', 'LC Expiry Date', 'LC Current Balance', 'LC Original Amount', \
#                 'Purchase Dry Weight', 'Purchase Wet Weight', 'Unpaid Provisional Receivables(USD)', 'Unpaid Final Receivables(USD)',
#                     'Uninvoiced Final Receivables(USD)', 'SCR Covered By LC (SCR-Sales Contractual Rights)', 'Total Financeable Sales Invoices Covered by LC', \
#                         'LC Cover Available to SCR (SCR-Sales Contractual Rights)', 'Sales Provisional Final', 'Sales Provisional Percentage', \
#                             'Sales Provisional Number', 'Financing Ratio', 'Sales Invoice Date', 'Sales Invoice Due Date', 'Sales Invoice Days Overdue', \
#                                 'Cost Code ID', 'Invoice Line Quantity', 'Invoice Line Quantity UOM', 'Purchase Dry Weight UOM', 'Purchase Wet Weight UOM', 'Receipt In Progress', \
#                                     'Delivery In Progress', 'Summary Outstanding Receivables(USD)', 'Sales Invoice Counterparty','Eligible Dry Weight', 'Eligible Wet Weight', 'Sales Assignment Eligible Percentage']

order = ['Purchase Group Company Code', 'Purchase Trade ID', 'Purchase Quota Titan ID', 'Purchase Assignment Reference', 'Sales Assignment Reference','Purchase Traffic Operator', \
    'Purchase Contractual Counterparty', 'Purchase Commodity', 'Sales Assignment Counterparty', 'Sales Credit Security Type', 'LC Reference', 'LC Original Amount', \
        'LC Current Balance', 'LC Issuance Date', 'LC Expiry Date', 'LC Opening Bank', 'LC Confirming Bank', 'Total Financeable Sales Invoices Covered by LC', \
            'LC Cover Available to SCR (SCR-Sales Contractual Rights)', 'SCR Covered By LC (SCR-Sales Contractual Rights)', 'Sales Invoice ID', 'Sales Provisional Final', \
                'Sales Provisional Number', 'Sales Provisional Percentage', 'Sales Invoice Date', 'Sales Invoice Due Date', 'Sales Invoice Days Overdue', 'Sales Invoice Counterparty', \
                    'Cost Code ID', 'Cost Amount(USD)', 'Invoice Line Quantity', 'Invoice Line Quantity UOM', 'Invoice Line Amount(USD)', 'Paid Amount', 'Sales Assignment Eligible Percentage', \
                        'Authorised Paid Quantity', 'Financed Quantity', 'Financing Ratio', 'Unpaid Provisional Receivables(USD)', 'Unpaid Final Receivables(USD)', 'Uninvoiced Final Receivables(USD)', \
                            'Summary Outstanding Receivables(USD)', 'Sales Contractual Counterparty Open Account Approved', 'Purchase Dry Weight', 'Purchase Dry Weight UOM', \
                                'Purchase Wet Weight', 'Purchase Wet Weight UOM', 'Receipt In Progress', 'Delivery In Progress', 'Eligible Dry Weight', 'Eligible Wet Weight']

final_sales_test = final_sales[order]

result4 = pd.merge(final_sales_test, df4, on=["Purchase Assignment Reference"], how='right')
# result4 = result4.drop_duplicates(['Purchase Assignment Reference'])

# import numpy as np
# test_p = result4.copy(deep=True)

# test_p['Purchase Contractual Counterparty'] = np.nan
# test_p['Sales Credit Security Type'] = np.nan
# test_p['LC Reference'] = np.nan
# test_p['LC Original Amount'] = np.nan
# test_p['LC Current Balance'] = np.nan
# test_p['LC Issuance Date'] = np.nan
# test_p['LC Expiry Date'] = np.nan
# test_p['LC Opening Bank'] = np.nan
# test_p['LC Confirming Bank'] = np.nan
# test_p['Total Financeable Sales Invoices Covered by LC'] = np.nan
# test_p['LC Cover Available to SCR (SCR-Sales Contractual Rights)'] = np.nan
# test_p['SCR Covered By LC (SCR-Sales Contractual Rights)'] = np.nan
# test_p['Cost Amount(USD)'] = np.nan
# test_p['Authorised Paid Quantity'] = np.nan
# test_p['Financed Quantity'] = np.nan
# test_p['Financing Ratio'] = np.nan
# test_p['Uninvoiced Final Receivables(USD)'] = np.nan
# test_p['Summary Outstanding Receivables(USD)'] = np.nan
# test_p['Sales Contractual Counterparty Open Account Approved '] = np.nan
# test_p['Purchase Dry Weight'] = np.nan
# test_p['Purchase Dry Weight UOM'] = np.nan
# test_p['Purchase Wet Weight'] = np.nan
# test_p['Purchase Wet Weight UOM'] = np.nan
# test_p['Receipt In Progress'] = np.nan
# test_p['Delivery In Progress'] = np.nan
# test_p['Eligible Dry Weight'] = np.nan
# test_p['Eligible Wet Weight'] = np.nan

# test_q = result4.copy(deep=True)
# test_q['Sales Invoice ID'] = np.nan
# test_q['Sales Provisional Final '] = np.nan
# test_q['Sales Provisional Number'] = np.nan
# test_q['Sales Provisional Percentage'] = np.nan
# test_q['Sales Invoice Date'] = np.nan
# test_q['Sales Invoice Due Date'] = np.nan
# test_q['Sales Invoice Days Overdue'] = np.nan
# test_q['Sales Invoice Counterparty'] = np.nan
# test_q['Cost Code ID'] = np.nan
# test_q['Invoice Line Quantity'] = np.nan
# test_q['Invoice Line Quantity UOM'] = np.nan

# test_f2 = pd.concat([test_p,test_q])
# test_f2 = test_f2.sort_values(by=['Purchase Assignment Reference'], ascending=False)

In [67]:
result4['Cost Amount(USD)'] = pd.to_numeric(result4['Cost Amount(USD)'], errors='coerce')
result4['Paid Amount'] = pd.to_numeric(result4['Paid Amount'], errors='coerce')

In [68]:
# group = result4.groupby('Sales Assignment Reference)('Sales Assignment Reference', as_index=False)
df1 = result4.groupby('Sales Assignment Reference')['Invoice Line Amount(USD)'].sum()
df2 = result4.groupby('Sales Assignment Reference')['Cost Amount(USD)'].sum()
df3 = result4.groupby('Sales Assignment Reference')['Paid Amount'].sum()
df1.reset_index(drop=False)
df2.reset_index(drop=False)
df3.reset_index(drop=False)


df4 = pd.merge(df1,df2, on = 'Sales Assignment Reference')
df5 = pd.merge(df4,df3, on = 'Sales Assignment Reference')

df5 = df5.reset_index()
df5

,Sales Assignment Reference,Invoice Line Amount(USD),Cost Amount(USD),Paid Amount
0,670639.12.1,9155085.83,32469392.16,0.00
1,670639.13.1,1757236.92,9367816.66,0.00


In [69]:
order = ['Purchase Group Company Code', 'Purchase Trade ID', 'Purchase Quota Titan ID', 'Purchase Assignment Reference', 'Sales Assignment Reference','Purchase Traffic Operator', \
    'Purchase Contractual Counterparty', 'Purchase Commodity', 'Sales Assignment Counterparty', 'Sales Credit Security Type', 'LC Reference', 'LC Original Amount', \
        'LC Current Balance', 'LC Issuance Date', 'LC Expiry Date', 'LC Opening Bank', 'LC Confirming Bank', 'Total Financeable Sales Invoices Covered by LC', \
            'LC Cover Available to SCR (SCR-Sales Contractual Rights)', 'SCR Covered By LC (SCR-Sales Contractual Rights)', 'Sales Invoice ID', 'Sales Provisional Final', \
                'Sales Provisional Number', 'Sales Provisional Percentage', 'Sales Invoice Date', 'Sales Invoice Due Date', 'Sales Invoice Days Overdue', 'Sales Invoice Counterparty', \
                    'Cost Code ID', 'Cost Amount(USD)', 'Invoice Line Quantity', 'Invoice Line Quantity UOM', 'Invoice Line Amount(USD)', 'Paid Amount', 'Sales Assignment Eligible Percentage', \
                        'Authorised Paid Quantity', 'Financed Quantity', 'Financing Ratio', 'Unpaid Provisional Receivables(USD)', 'Unpaid Final Receivables(USD)', 'Uninvoiced Final Receivables(USD)', \
                            'Summary Outstanding Receivables(USD)', 'Sales Contractual Counterparty Open Account Approved', 'Purchase Dry Weight', 'Purchase Dry Weight UOM', \
                                'Purchase Wet Weight', 'Purchase Wet Weight UOM', 'Receipt In Progress', 'Delivery In Progress', 'Eligible Dry Weight', 'Eligible Wet Weight']
                                
dff = pd.merge(df5, result4, on='Sales Assignment Reference')
dff = dff.drop(columns = ['Cost Amount(USD)_y', 'Paid Amount_y', 'Invoice Line Amount(USD)_y'])
dff = dff.drop_duplicates(['Purchase Assignment Reference'])
dff = dff.rename(columns={'Cost Amount(USD)_x':'Cost Amount(USD)', 'Paid Amount_x':'Paid Amount', 'Invoice Line Amount(USD)_x':'Invoice Line Amount(USD)'})
dff = dff [order]
dff

,Purchase Group Company Code,Purchase Trade ID,Purchase Quota Titan ID,Purchase Assignment Reference,Sales Assignment Reference,Purchase Traffic Operator,Purchase Contractual Counterparty,Purchase Commodity,Sales Assignment Counterparty,Sales Credit Security Type,...,Summary Outstanding Receivables(USD),Sales Contractual Counterparty Open Account Approved,Purchase Dry Weight,Purchase Dry Weight UOM,Purchase Wet Weight,Purchase Wet Weight UOM,Receipt In Progress,Delivery In Progress,Eligible Dry Weight,Eligible Wet Weight
0,PTE,669658.00,669658.18,669658.18.1,670639.12.1,Ajit Braganza,Citic Metal HK Limited,Copper Concentrates,Konkola Copper Mines PLC,NaN,...,16234696.08,N,1512.01,MT,1684.85,MT,N,N,1512.01,1684.85
2,PTE,669658.00,669658.21,669658.21.1,670639.13.1,Rebecca Pu,Citic Metal HK Limited,Copper Concentrates,Konkola Copper Mines PLC,NaN,...,9367816.66,N,2146.00,MT,2395.88,MT,N,N,2146.00,2395.88


In [70]:
import numpy as np

dff['Sales Invoice ID'] = np.nan
dff['Sales Provisional Final '] = np.nan
dff['Sales Provisional Number'] = np.nan
dff['Sales Provisional Percentage'] = np.nan
dff['Sales Invoice Date'] = np.nan
dff['Sales Invoice Due Date'] = np.nan
dff['Sales Invoice Days Overdue'] = np.nan
dff['Sales Invoice Counterparty'] = np.nan
dff['Cost Code ID'] = np.nan
dff['Invoice Line Quantity'] = np.nan
dff['Invoice Line Quantity UOM'] = np.nan




In [75]:
dff['Total Financeable Sales Invoices Covered by LC']  = dff.apply(lambda x: function3(x['Invoice Line Amount(USD)'], x['LC Current Balance']), axis = 1)
dff['LC Cover Available to SCR (SCR-Sales Contractual Rights)']  = dff.apply(lambda x: function4(x['Invoice Line Amount(USD)'], x['LC Current Balance']), axis = 1)
dff['Sales Provisional Percentage']  = dff.apply(lambda x: function6(x['Sales Provisional Final ']), axis = 1)
dff['LC Cover Available to SCR (SCR-Sales Contractual Rights)'] = pd.to_numeric(dff['LC Cover Available to SCR (SCR-Sales Contractual Rights)'], errors='coerce') #chenge to float
dff['SCR Covered By LC (SCR-Sales Contractual Rights)']  = dff.apply(lambda x: function5(x['LC Cover Available to SCR (SCR-Sales Contractual Rights)'], x['Uninvoiced Final Receivables(USD)']), axis = 1)

In [73]:
result4['LC Current Balance'] = np.nan
result4['Sales Credit Security Type'] = np.nan
result4['LC Reference'] = np.nan
result4['LC Original Amount'] = np.nan
result4['LC Issuance Date'] = np.nan
result4['LC Expiry Date'] = np.nan
result4['LC Confirming Bank'] = np.nan
result4['Total Financeable Sales Invoices Covered by LC'] = np.nan
result4['LC Cover Available to SCR (SCR-Sales Contractual Rights)'] = np.nan
result4['SCR Covered By LC (SCR-Sales Contractual Rights)'] = np.nan
result4['Cost Amount(USD)'] = np.nan
result4['Authorized Paid Quantity'] = np.nan
result4['Financed Quantity'] = np.nan
result4['Uninvoiced Final Receivables(USD)'] = np.nan
result4['Summary Outstanding Receivables(USD)'] = np.nan
result4['Sales Contractual Counterparty Open Account Approved '] = np.nan
result4['Purchase Dry Weight'] = np.nan
result4['Purchase Dry Weight UOM'] = np.nan
result4['Purchase Wet Weight'] = np.nan
result4['Purchase Wet Weight UOM'] = np.nan
result4['Receipt In Progress'] = np.nan
result4['Delivery In Progress'] = np.nan
result4['Eligible Dry Weight'] = np.nan
result4['Eligible Wet Weight'] = np.nan

In [77]:
final_result = pd.concat([dff, result4], ignore_index=True)
final_result = final_result.sort_values(by=['Sales Assignment Reference', 'Delivery In Progress'], ascending=[False, False], na_position= 'first') 
# final_result = final_result[order]


final_result = final_result[order]
final_result

,Purchase Group Company Code,Purchase Trade ID,Purchase Quota Titan ID,Purchase Assignment Reference,Sales Assignment Reference,Purchase Traffic Operator,Purchase Contractual Counterparty,Purchase Commodity,Sales Assignment Counterparty,Sales Credit Security Type,...,Summary Outstanding Receivables(USD),Sales Contractual Counterparty Open Account Approved,Purchase Dry Weight,Purchase Dry Weight UOM,Purchase Wet Weight,Purchase Wet Weight UOM,Receipt In Progress,Delivery In Progress,Eligible Dry Weight,Eligible Wet Weight
4,PTE,669658.00,669658.21,669658.21.1,670639.13.1,Rebecca Pu,Citic Metal HK Limited,Copper Concentrates,Konkola Copper Mines PLC,NaN,...,NaN,N,nan,NaN,nan,NaN,NaN,NaN,nan,nan
1,PTE,669658.00,669658.21,669658.21.1,670639.13.1,Rebecca Pu,Citic Metal HK Limited,Copper Concentrates,Konkola Copper Mines PLC,NaN,...,9367816.66,N,2146.00,MT,2395.88,MT,N,N,2146.00,2395.88
2,PTE,669658.00,669658.18,669658.18.1,670639.12.1,Ajit Braganza,Citic Metal HK Limited,Copper Concentrates,Konkola Copper Mines PLC,NaN,...,NaN,N,nan,NaN,nan,NaN,NaN,NaN,nan,nan
3,PTE,669658.00,669658.18,669658.18.1,670639.12.1,Ajit Braganza,Citic Metal HK Limited,Copper Concentrates,Konkola Copper Mines PLC,NaN,...,NaN,N,nan,NaN,nan,NaN,NaN,NaN,nan,nan
0,PTE,669658.00,669658.18,669658.18.1,670639.12.1,Ajit Braganza,Citic Metal HK Limited,Copper Concentrates,Konkola Copper Mines PLC,NaN,...,16234696.08,N,1512.01,MT,1684.85,MT,N,N,1512.01,1684.85


: 

In [31]:
writer = pd.ExcelWriter(r"C:\Users\Yuyan.Lu\Desktop\BB REPORT TEST FILE\0118\final_test.xlsx", engine='openpyxl')
book = load_workbook(writer.path)
writer.book = book
final_result.to_excel(excel_writer=writer, sheet_name='sales_receivables')
writer.save()
writer.close()


In [32]:
final_sales_test.to_excel(r"C:\Users\Yuyan.Lu\Desktop\BB REPORT TEST FILE\0118\test10.xlsx")